# 라이브러리

In [1]:
options(warn =-1, repr.plot.width=15, repr.plot.height=9)
# ## 패키지 설치
library(dplyr)
library(stringr)
library(ggplot2)
library(gridExtra)
library(scales)
library(randomForest)
library(lubridate)
library(caret)
library(e1071)
library(nnet)
library(class)



다음의 패키지를 부착합니다: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



다음의 패키지를 부착합니다: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


다음의 패키지를 부착합니다: ‘randomForest’


The following object is masked from ‘package:gridExtra’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine



다음의 패키지를 부착합니다: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


필요한 패키지를 로딩중입니다: lattice



# 전처리

In [2]:
### .csv 불러오기
data <- read.csv("./data/refined_data_filtered_major5_subset.csv")

In [3]:
data$근무지역분류 <- as.character(data$근무지역분류)
data$근무지역분류 <- factor(data$근무지역분류)
data$장애유형 <- as.character(data$장애유형)
data$장애유형 <- factor(data$장애유형)
data$중증여부 <- as.character(data$중증여부)
data$중증여부 <- factor(data$중증여부)
data$근무지역 <- as.character(data$근무지역)
data$근무지역 <- factor(data$근무지역)
data$취업직종대분류 <- as.character(data$취업직종대분류)
data$취업직종대분류 <- factor(data$취업직종대분류)
data$연령대 <- as.character(data$연령대)
data$연령대 <- factor(data$연령대)
data$장애유형_중증여부 <- as.character(data$장애유형_중증여부)
data$장애유형_중증여부 <- factor(data$장애유형_중증여부)
data$월 <- as.character(data$월)
data$월 <- factor(data$월)

In [4]:
data$target<-(ifelse(data$취업직종대분류=="경영·행정·사무직","경영·행정·사무직",data$취업직종대분류))

data$target <- ifelse(data$취업직종대분류 %in% c("청소 및 기타 개인서비스직", "음식 서비스직", "미용·예식 서비스직", "영업·판매직", "스포츠·레크리에이션직","경호·경비직"), "서비스직", data$target)

data$target <- ifelse(grepl("^\\d+$", data$target), "기타", data$target)

In [5]:
table(data$target)


경영·행정·사무직             기타         서비스직 
            8961             7474             8544 

In [6]:
data <- subset(data, data$target == "기타")
table(data$target)


기타 
7474 

In [7]:
data$target<- ifelse(data$취업직종대분류=="제조 단순직","제조 단순직",data$취업직종대분류)
data$target <- ifelse(data$취업직종대분류 %in% c("보건·의료직","돌봄 서비스직(간병·육아)","사회복지·종교직"), "간호·보건 및 돌봄 서비스 관련 직군", data$target)
data$target <- ifelse(data$target %in% c("간호·보건 및 돌봄 서비스 관련 직군","제조 단순직"),  data$target,"전문·생산 및 정비 관련 직군")
table(data$target)


간호·보건 및 돌봄 서비스 관련 직군        전문·생산 및 정비 관련 직군 
                              2291                               2709 
                       제조 단순직 
                              2474 

In [8]:
data <- subset(data, data$target == "전문·생산 및 정비 관련 직군")
table(data$target)


전문·생산 및 정비 관련 직군 
                       2709 

In [9]:
data$target<- ifelse(data$취업직종대분류=="인쇄·목재·공예 및 기타 설치·정비·생산직","인쇄·목재·공예 및 기타 설치·정비·생산직",data$취업직종대분류)
data$target <- ifelse(data$취업직종대분류 %in% c("전기·전자 설치·정비·생산직"), "전기·전자 설치·정비·생산직", data$target)
data$target <- ifelse(data$취업직종대분류 %in% c("식품 가공·생산직"), "식품 가공·생산직", data$target)
data$target <- ifelse(data$취업직종대분류 %in% c("기계 설치·정비·생산직"), "기계 설치·정비·생산직", data$target)
data$target <- ifelse(data$취업직종대분류 %in% c("섬유·의복 생산직"), "섬유·의복 생산직", data$target)
data$target <- ifelse(data$취업직종대분류 %in% c("건설·채굴직"), "건설·채굴직", data$target)
data$target <- ifelse(data$취업직종대분류 %in% c("예술·디자인·방송직"), "예술·디자인·방송직", data$target)
data$target <- ifelse(data$취업직종대분류 %in% c("운전·운송직"), "운전·운송직", data$target)
data$target <- ifelse(data$취업직종대분류 %in% c("교육직"), "교육직", data$target)
data$target <- ifelse(data$취업직종대분류 %in% c("관리직(임원·부서장)"), "관리직(임원·부서장)", data$target)
table(data$target)


                            건설·채굴직                     관리직(임원·부서장) 
                                    134                                     175 
                                 교육직                   기계 설치·정비·생산직 
                                    174                                     243 
                       섬유·의복 생산직                        식품 가공·생산직 
                                    182                                     276 
                     예술·디자인·방송직                             운전·운송직 
                                    135                                     349 
인쇄·목재·공예 및 기타 설치·정비·생산직              전기·전자 설치·정비·생산직 
                                    680                                     361 

In [10]:
# 불필요한 컬럼 제거

data_filtered <- subset(data, select = -c(취업일자,연령,장애유형,중증여부,근무지역))
sum(is.na(data_filtered))

[1] 0

In [11]:
# 연령대
data_filtered$age_20 <- data_filtered$연령대 == 1
data_filtered$age_30 <- data_filtered$연령대 == 2
data_filtered$age_40 <- data_filtered$연령대 == 3
data_filtered$age_50 <- data_filtered$연령대 == 4
data_filtered$age_60 <- data_filtered$연령대 == 5
data_filtered$age_70 <- data_filtered$연령대 == 6

In [12]:
# 장애유형_중증여부 바꾸기
data_filtered$시각장애_중증 <- data_filtered$장애유형_중증여부 == "시각장애-중증"
data_filtered$시각장애_경증 <- data_filtered$장애유형_중증여부 == "시각장애-경증"

data_filtered$지체장애_중증 <- data_filtered$장애유형_중증여부 == "지체장애-중증"
data_filtered$지체장애_경증 <- data_filtered$장애유형_중증여부 == "지체장애-경증"

data_filtered$지적장애_경증 <- data_filtered$장애유형_중증여부 == "지적장애-경증"
data_filtered$지적장애_중증 <- data_filtered$장애유형_중증여부 == "지적장애-중증"

data_filtered$청각장애_경증 <- data_filtered$장애유형_중증여부 == "청각장애-경증"
data_filtered$청각장애_중증 <- data_filtered$장애유형_중증여부 == "청각장애-중증"

data_filtered$정신장애_경증 <- data_filtered$장애유형_중증여부 == "정신장애-경증"
data_filtered$정신장애_중증 <- data_filtered$장애유형_중증여부 == "정신장애-중증"

In [13]:
#   근무지역 분류 바꾸기
data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

data_filtered$강원 <- data_filtered$근무지역분류 == "강원"

data_filtered$전남 <- data_filtered$근무지역분류 == "전남"

data_filtered$충북 <- data_filtered$근무지역분류 == "충북"

data_filtered$경기 <- data_filtered$근무지역분류 == "경기"

data_filtered$인천 <- data_filtered$근무지역분류 == "인천"

data_filtered$서울 <- data_filtered$근무지역분류 == "서울"

data_filtered$울산 <- data_filtered$근무지역분류 == "울산"

data_filtered$대전 <- data_filtered$근무지역분류 == "대전"

data_filtered$부산 <- data_filtered$근무지역분류 == "부산"

data_filtered$전북 <- data_filtered$근무지역분류 == "전북"

data_filtered$광주 <- data_filtered$근무지역분류 == "광주"

data_filtered$충남 <- data_filtered$근무지역분류 == "충남"

data_filtered$대구 <- data_filtered$근무지역분류 == "대구"

data_filtered$경남 <- data_filtered$근무지역분류 == "경남"

data_filtered$제주 <- data_filtered$근무지역분류 == "제주"

data_filtered$세종 <- data_filtered$근무지역분류 == "세종"

data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

In [14]:
# 월
data_filtered$Jan <- data_filtered$월  == 01
data_filtered$Feb <- data_filtered$월  == 02
data_filtered$Mar <- data_filtered$월  == 03
data_filtered$Apr <- data_filtered$월  == 04
data_filtered$May <- data_filtered$월  == 05
data_filtered$Jun <- data_filtered$월  == 06
data_filtered$Jul <- data_filtered$월  == 07
data_filtered$Aug <- data_filtered$월  == 08
data_filtered$Sep <- data_filtered$월  == 09
data_filtered$Oct <- data_filtered$월  == 10
data_filtered$Nov <- data_filtered$월  == 11
data_filtered$Dec <- data_filtered$월  == 12

In [15]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	2709 obs. of  51 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 19 16 19 16 16 19 15 18 18 18 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 6 4 3 3 3 2 1 1 1 1 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 7 7 7 7 6 7 6 6 6 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 1 10 5 7 7 8 10 3 2 2 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 1 9 2 2 2 2 9 17 17 17 ...
 $ target           : chr  "전기·전자 설치·정비·생산직" "운전·운송직" "전기·전자 설치·정비·생산직" "운전·운송직" ...
 $ age_20           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30           : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ age_40           : logi  FALSE FALSE TRUE TRUE TRUE FALSE ...
 $ age_50           : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70           : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증  

[1] 0

In [16]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	2709 obs. of  51 variables:
 $ 취업직종대분류   : Factor w/ 21 levels "건설·채굴직",..: 19 16 19 16 16 19 15 18 18 18 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 6 4 3 3 3 2 1 1 1 1 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 7 7 7 7 6 7 6 6 6 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 1 10 5 7 7 8 10 3 2 2 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 1 9 2 2 2 2 9 17 17 17 ...
 $ target           : chr  "전기·전자 설치·정비·생산직" "운전·운송직" "전기·전자 설치·정비·생산직" "운전·운송직" ...
 $ age_20           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30           : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ age_40           : logi  FALSE FALSE TRUE TRUE TRUE FALSE ...
 $ age_50           : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70           : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증  

[1] 0

In [17]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [18]:
str(data_filtered_subset)
sum(is.na(data_filtered_subset))

'data.frame':	2709 obs. of  47 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 19 16 19 16 16 19 15 18 18 18 ...
 $ target        : chr  "전기·전자 설치·정비·생산직" "운전·운송직" "전기·전자 설치·정비·생산직" "운전·운송직" ...
 $ age_20        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30        : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ age_40        : logi  FALSE FALSE TRUE TRUE TRUE FALSE ...
 $ age_50        : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70        : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증 : logi  FALSE TRUE TRUE TRUE TRUE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장

[1] 0

In [19]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [20]:
str(data_filtered_subset)

'data.frame':	2709 obs. of  47 variables:
 $ 취업직종대분류: Factor w/ 21 levels "건설·채굴직",..: 19 16 19 16 16 19 15 18 18 18 ...
 $ target        : chr  "전기·전자 설치·정비·생산직" "운전·운송직" "전기·전자 설치·정비·생산직" "운전·운송직" ...
 $ age_20        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30        : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ age_40        : logi  FALSE FALSE TRUE TRUE TRUE FALSE ...
 $ age_50        : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70        : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증 : logi  FALSE TRUE TRUE TRUE TRUE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장

In [21]:
summary(data_filtered_subset)

                                 취업직종대분류    target         
 인쇄·목재·공예 및 기타 설치·정비·생산직:680    Length:2709       
 전기·전자 설치·정비·생산직             :361    Class :character  
 운전·운송직                            :349    Mode  :character  
 식품 가공·생산직                       :276                      
 기계 설치·정비·생산직                  :243                      
 섬유·의복 생산직                       :182                      
 (Other)                                :618                      
   age_20          age_30          age_40          age_50       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:1738      FALSE:2172      FALSE:2262      FALSE:2290     
 TRUE :971       TRUE :537       TRUE :447       TRUE :419      
                                                                
                                                                
                                                                
                                                                
   age

# 변동사항

In [22]:
data_filtered_subset$target <- factor(data_filtered_subset$target)

In [23]:
data_filtered_subset <- subset(data_filtered_subset, select = -c(취업직종대분류))

In [24]:
str(data_filtered_subset)

'data.frame':	2709 obs. of  46 variables:
 $ target       : Factor w/ 10 levels "건설·채굴직",..: 10 8 10 8 8 10 7 9 9 9 ...
 $ age_20       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_30       : logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ age_40       : logi  FALSE FALSE TRUE TRUE TRUE FALSE ...
 $ age_50       : logi  FALSE TRUE FALSE FALSE FALSE FALSE ...
 $ age_60       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_70       : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증: logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_경증: logi  FALSE TRUE TRUE TRUE TRUE FALSE ...
 $ 지적장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증: logi  FALSE FALSE FALSE FALSE FALSE TRUE ...
 $ 청각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 정신장애_경증: logi  FALSE FALSE FALSE FALSE FA

In [25]:
lm(target~.,data_filtered_subset)


Call:
lm(formula = target ~ ., data = data_filtered_subset)

Coefficients:
      (Intercept)         age_20TRUE         age_30TRUE         age_40TRUE  
          6.54993            0.82226            1.31925            1.12374  
       age_50TRUE         age_60TRUE         age_70TRUE  시각장애_중증TRUE  
          0.98441            0.83259                 NA           -2.12395  
시각장애_경증TRUE  지체장애_중증TRUE  지체장애_경증TRUE  지적장애_경증TRUE  
         -1.42834           -0.74335           -1.02189           -3.55808  
지적장애_중증TRUE  청각장애_경증TRUE  청각장애_중증TRUE  정신장애_경증TRUE  
          0.25070           -0.81343           -0.11427            0.66471  
정신장애_중증TRUE           경북TRUE           강원TRUE           전남TRUE  
               NA            0.42243           -0.64508           -0.80855  
         충북TRUE           경기TRUE           인천TRUE           서울TRUE  
          0.60180            0.56090            0.25936           -0.19064  
         울산TRUE           대전TRUE           부산TRUE           전북TRUE  
     

In [26]:
summary(data_filtered_subset)

                                     target      age_20          age_30       
 인쇄·목재·공예 및 기타 설치·정비·생산직:680   Mode :logical   Mode :logical  
 전기·전자 설치·정비·생산직             :361   FALSE:1738      FALSE:2172     
 운전·운송직                            :349   TRUE :971       TRUE :537      
 식품 가공·생산직                       :276                                  
 기계 설치·정비·생산직                  :243                                  
 섬유·의복 생산직                       :182                                  
 (Other)                                :618                                  
   age_40          age_50          age_60          age_70       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:2262      FALSE:2290      FALSE:2416      FALSE:2667     
 TRUE :447       TRUE :419       TRUE :293       TRUE :42       
                                                                
                                                                
                                 

# 시험준비

In [27]:
samp <- createDataPartition(data_filtered_subset$target, p=0.7, list=F)

#   학습용
data.tr <- data_filtered_subset[samp,]

#   검증용
data.test <- data_filtered_subset[-samp,]

In [28]:
#   KNN을 위한 Training Data 준비
data.training <- subset(data.tr, select = -target)
data.training.label <- data.tr$target

In [29]:
#   문제 및 정답 만들기
x   <- subset(data.test, select = -target)
y   <- data.test$target


# 학습시키기

In [30]:
#  랜덤포레스트 학습 시키기
rf <- randomForest(target ~., data.tr, ntree=50)

In [31]:
# SVM 학습시키기
modelsvm <- svm(target~.,data.tr)

In [32]:
# KNN 학습시키기 3,5,7,
knnmodel <- knn(train=data.training, test=x,cl=data.training.label,k=3)

In [33]:
# nnet 학습시키기
modelnnet <- nnet(target~.,data.tr,size =10)

# weights:  570
initial  value 4847.524368 
iter  10 value 3578.670123
iter  20 value 3051.333971
iter  30 value 2856.415420
iter  40 value 2734.397988
iter  50 value 2652.372026
iter  60 value 2605.167320
iter  70 value 2564.976697
iter  80 value 2527.961449
iter  90 value 2503.881213
iter 100 value 2482.108833
final  value 2482.108833 
stopped after 100 iterations


# 검증하기

In [34]:
#  Random Forest 검증하기
predrf <- predict(rf, x)

In [35]:
#  SVM 검증하기
predmodelsvm <- predict(modelsvm, x)

In [36]:
# KNN 검증하기
# mean(knnmodel==y) 

In [37]:
# nnet 검증하기
prednnet = predict(modelnnet, x,type = "class")    # class : 객관식, raw : 주관식

# 예측력 확인

In [38]:
#  RF 예측력
mean(predrf == y)

[1] 0.480198

In [39]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.3688119

In [40]:
#  KNN 예측력
mean(knnmodel == y)

[1] 0.4430693

In [41]:
#  nnet 예측력
mean(prednnet == y)

[1] 0.4282178

# 오분류표

In [42]:
#  Random Forest 오분류표
table(predrf,y)

                                         y
predrf                                    건설·채굴직 관리직(임원·부서장)
  건설·채굴직                                       9                   4
  관리직(임원·부서장)                               4                   5
  교육직                                            1                   4
  기계 설치·정비·생산직                             1                   7
  섬유·의복 생산직                                  2                   1
  식품 가공·생산직                                  0                   4
  예술·디자인·방송직                                0                   0
  운전·운송직                                      13                  16
  인쇄·목재·공예 및 기타 설치·정비·생산직           4                   8
  전기·전자 설치·정비·생산직                        6                   3
                                         y
predrf                                    교육직 기계 설치·정비·생산직
  건설·채굴직                                  0                     4
  관리직(임원·부서장)                          1                     0
  교육

In [43]:
#  SVM 오분류표
table(predmodelsvm,y)

                                         y
predmodelsvm                              건설·채굴직 관리직(임원·부서장)
  건설·채굴직                                       0                   0
  관리직(임원·부서장)                               0                   0
  교육직                                            1                   1
  기계 설치·정비·생산직                             0                   0
  섬유·의복 생산직                                  0                   0
  식품 가공·생산직                                  0                   1
  예술·디자인·방송직                                0                   0
  운전·운송직                                      29                  33
  인쇄·목재·공예 및 기타 설치·정비·생산직           8                  14
  전기·전자 설치·정비·생산직                        2                   3
                                         y
predmodelsvm                              교육직 기계 설치·정비·생산직
  건설·채굴직                                  0                     0
  관리직(임원·부서장)                          0                     0
  교육

In [44]:
#  KNN 오분류표
# table(y, knnmodel)

In [45]:
# nnet 오분류표
table(prednnet,y)

                                         y
prednnet                                  건설·채굴직 관리직(임원·부서장)
  건설·채굴직                                      11                  10
  관리직(임원·부서장)                               7                   5
  교육직                                            2                   2
  기계 설치·정비·생산직                             1                   3
  섬유·의복 생산직                                  0                   3
  식품 가공·생산직                                  0                   4
  예술·디자인·방송직                                4                   4
  운전·운송직                                       5                  10
  인쇄·목재·공예 및 기타 설치·정비·생산직           4                   6
  전기·전자 설치·정비·생산직                        6                   5
                                         y
prednnet                                  교육직 기계 설치·정비·생산직
  건설·채굴직                                  1                     6
  관리직(임원·부서장)                          2                     3
  교육

In [46]:
# 새로운 데이터에 대한 예측
predict(rf, (list(age_20=FALSE,age_30=FALSE,age_40=FALSE,age_50=FALSE,age_60=TRUE,age_70=FALSE,
            시각장애_중증=TRUE,시각장애_경증=FALSE,지체장애_경증=FALSE,지체장애_중증=FALSE,지적장애_경증=FALSE,지적장애_중증=FALSE,
            청각장애_경증=FALSE,청각장애_중증=FALSE,정신장애_경증=FALSE,정신장애_중증=FALSE,
            경북=TRUE,강원=FALSE,전남=FALSE,충북=FALSE,경기=FALSE,인천=FALSE,서울=FALSE,울산=FALSE,대전=FALSE,부산=FALSE,
            전북=FALSE,광주=FALSE,충남=FALSE,대구=FALSE,경남=FALSE,제주=FALSE,세종=FALSE,경북=FALSE,
            Jan=TRUE,Feb=FALSE,Mar=FALSE,Apr=FALSE,May=FALSE,Jun=FALSE,Jul=FALSE,Aug=FALSE,Sep=FALSE,
            Oct=FALSE,Nov=FALSE,Dec=FALSE)), type = "prob")

# pred_prob <- predict(rf, new_data, type = "prob")
# pred_prob <- predict(rf, new_data, type = "prob")

,건설·채굴직,관리직(임원·부서장),교육직,기계 설치·정비·생산직,섬유·의복 생산직,식품 가공·생산직,예술·디자인·방송직,운전·운송직,인쇄·목재·공예 및 기타 설치·정비·생산직,전기·전자 설치·정비·생산직
1,0.04,0.1,0.06,0.22,0,0,0,0.02,0.54,0.02
